<a href="https://colab.research.google.com/github/RogerHeederer/DL_Entry_Pytorch/blob/master/MakeWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fde4c0b3710>)

훈련용 데이터 전처리

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [9]:
parse_text[:40]

'Here are two reasons companies fail: the'

In [10]:
#(Audio), (Laugter) 등 배경음 표시한 텍스트들 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [11]:
# 문장 토큰화
sent_text = sent_tokenize(content_text)

In [14]:
sent_text[:3]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.',
 'Both are necessary, but it can be too much of a good thing.']

In [15]:
normalized_text = []
# 각 문장 별 구두 점 제거 + 소문자 전환
for string in sent_text:
  tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
  normalized_text.append(tokens)

In [16]:
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 워드 토크나이즈 수행

In [18]:
print('총 샘풀의 개수 : {}'.format(len(result)))

총 샘풀의 개수 : 273424


In [19]:
type(result)

list

In [21]:
for line in result[:3]:
  print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


#해당 단어들 워드 투 백 훈련 시키기

In [22]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)
#size : 임베딩 후 벡터 차원
#window : 컨텍스트 윈도우 크기
#min_count : 단어 최소 빈도 수 제한
#workers : 학습을 위한 프로세스 숫자
#sg : 0은 CBOW, 1은 Skip-gram

In [23]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8474330902099609), ('guy', 0.8181765079498291), ('boy', 0.7752984762191772), ('girl', 0.7622597813606262), ('lady', 0.7586252689361572), ('soldier', 0.7582021951675415), ('kid', 0.7162522673606873), ('gentleman', 0.7124583721160889), ('friend', 0.6697666645050049), ('physicist', 0.6584862470626831)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


#학습한 모델 저장하기

In [24]:
model.wv.save_word2vec_format('./eng_w2v') #모델 저장

NameError: ignored

In [27]:
from gensim.models import KeyedVectors

In [28]:
loaded_model = KeyedVectors.load_word2vec_format("./eng_w2v") # 모델 로드

In [30]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8474330902099609), ('guy', 0.8181765079498291), ('boy', 0.7752984762191772), ('girl', 0.7622597813606262), ('lady', 0.7586252689361572), ('soldier', 0.7582021951675415), ('kid', 0.7162522673606873), ('gentleman', 0.7124583721160889), ('friend', 0.6697666645050049), ('physicist', 0.6584862470626831)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
